Considering a system using machine learning to detect spam SMS text messages. Our ML systems workflow is like this: Train offline -> Make model available as a service -> Predict online.
    A classifier is trained offline with spam and non-spam messages.
    The trained model is deployed as a service to serve users.

When we develop a machine learning model, we need to think about how to deploy it, that is, how to make this model available to other users.

we will focus on both: building a machine learning model for spam SMS message classification, then create an API for the model, using Flask, the Python micro framework for building web applications.This API allows us to utilize the predictive capabilities through HTTP requests. Let’s get started!

The data is a collection of SMS messages tagged as spam or ham that can be found [here](https://www.kaggle.com/uciml/sms-spam-collection-dataset). First, we will use this dataset to build a prediction model that will accurately classify which texts are spam.

Naive Bayes classifiers are a popular statistical technique of e-mail filtering. They typically use bag of words features to identify spam e-mail. Therefore, We’ll build a simple message classifier using Naive Bayes theorem.

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [0]:
data=pd.read_csv("./spam.csv", encoding="latin-1")

In [70]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
data["label"]=data["v1"].map({"ham":0, "spam":1})

In [72]:
data.head()

,v1,v2,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [0]:
X=data["v2"]
y=data["label"]
count_vect=CountVectorizer()
X=count_vect.fit_transform(X)

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.33,random_state=42)

In [75]:
#model
class_naive=MultinomialNB()
class_naive.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [76]:
class_naive.score(X_test,y_test)

0.9793365959760739

In [0]:
y_pred=class_naive.predict(X_test)

In [78]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1587
           1       0.93      0.92      0.92       252

   micro avg       0.98      0.98      0.98      1839
   macro avg       0.96      0.95      0.96      1839
weighted avg       0.98      0.98      0.98      1839



Not only Naive Bayes classifier is easy to implement but also provides very good result.

After training the model, it is desirable to have a way to persist the model for future use without having to retrain. To achieve this, we add the following lines to save our model as a .pkl file for the later use.

In [79]:
from sklearn.externals import joblib
joblib.dump(class_naive,"NB_spam_mdel.pkl")

['NB_spam_mdel.pkl']

And we can load and use saved model later like s

In [0]:
NB_spam_model=open("NB_spam_mdel.pkl","rb")
clf=joblib.load(NB_spam_model)

The above process called “persist model in a standard format”, that is, models are persisted in a certain format specific to the language in development.

And the model will be served in a micro-service that expose endpoints to receive requests from client. This is the next step.


**Turning the Spam Message Classifier into a Web Application**
Having prepared the code for classifying SMS messages in the previous section, we will develop a web application that consists of a simple web page with a form field that lets us enter a message. After submitting the message to the web application, it will render it on a new page which gives us a result of spam or not spam.

First, we create a folder for this project called SMS-Message-Spam-Detector , this is the directory tree inside the folder. We will explain each file.



```
spam.csv
app.py
templates/
        home.html
        result.html
static/
        style.css
```



The sub-directory templates is the directory in which Flask will look for static HTML files for rendering in the web browser, in our case, we have two html files: home.html and result.html .


**app.py**

The app.py file contains the main code that will be executed by the Python interpreter to run the Flask web application, it included the ML code for classifying SMS messages:

In [0]:
from flask import Flask,render_template,url_for, request
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

In [0]:
app= Flask(__name__)
@app.route("/")

def home():
  return render_template("home.html")

@app.route("/predict",methods=["POST"])
def predict():
  #Alternative Usage of Saved Model
  joblib.dump(clf, 'NB_spam_model.pkl')
  NB_spam_model = open('NB_spam_model.pkl','rb')
  class_naive = joblib.load(NB_spam_model)
  
  if request.method== "POST":
    message=request.form["message"]
    data[message]
    count_vect=class_naive.transform(data).toarray()
    my_prediction=class_naive.predict(vect)
  return render_template("results.html",prediction=my_prediction)

if "_name__"=="__main__":
  app.run(debug=Ture)



*    We ran our application as a single module; thus we initialized a new Flask instance with the argument __name__ to let Flask know that it can find the HTML template folder (templates) in the same directory where it is located.
*       Next, we used the route decorator (@app.route('/')) to specify the URL that should trigger the execution of the home function.

*       Our home function simply rendered the home.html HTML file, which is located in the templates folder.
  
*     Inside the predict function, we access the spam data set, pre-process the text, and make predictions, then store the model. We access the new message entered by the user and use our model to make a prediction for its label.

*       we used the POST method to transport the form data to the server in the message body. Finally, by setting the debug=True argument inside the app.run method, we further activated Flask's debugger.
*    Lastly, we used the run function to only run the application on the server when this script is directly executed by the Python interpreter, which we ensured using the if statement with __name__ == '__main__'.





   

**home.html**



```
<!DOCTYPE html>
<html>
<head>
	<title>Home</title>
	<!-- <link rel="stylesheet" type="text/css" href="../static/css/styles.css"> -->
	<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/styles.css') }}">
</head>
<body>

	<header>
		<div class="container">
		<div id="brandname">
			Machine Learning App with Flask
		</div>
		<h2>Spam Detector For SMS Messages</h2>
		
	</div>
	</header>
	<div class="ml-container">
		<form action="{{ url_for('predict')}}" method="POST">
		<p>Enter Your Message Here</p>
		<!-- <input type="text" name="comment"/> -->
		<textarea name="message" rows="4" cols="50"></textarea>
		<br/>
		<input type="submit" class="btn-info" value="predict">
	</form>
	</div>
</body>
</html>
```



**style.css**


In the header section of home.html, we loaded styles.cssfile. CSS is to determine how the look and feel of HTML documents. styles.css has to be saved in a sub-directory calledstatic, which is the default directory where Flask looks for static files such as CSS.
```
body{
	font:15px/1.5 Arial, Helvetica,sans-serif;
	padding: 0px;
	background-color:#f4f3f3;
}

.container{
	width:100%;
	margin: auto;
	overflow: hidden;
}

header{
	background:#03A9F4;#35434a;
	border-bottom:#448AFF 3px solid;
	height:120px;
	width:100%;
	padding-top:30px;

}

.main-header{
			text-align:center;
			background-color: blue;
			height:100px;
			width:100%;
			margin:0px;
		}
#brandname{
	float:left;
	font-size:30px;
	color: #fff;
	margin: 10px;
}

header h2{
	text-align:center;
	color:#fff;

}
.btn-info {background-color: #2196F3;
	height:40px;
	width:100px;} /* Blue */
.btn-info:hover {background: #0b7dda;}

.resultss{
	border-radius: 15px 50px;
    background: #345fe4;
    padding: 20px; 
    width: 200px;
    height: 150px;
}
```



**result.html**

we create a result.html file that will be rendered via the render_template('result.html', prediction=my_prediction) line return inside the predictfunction, which we defined in the app.py script to display the text that a user submitted via the text field. The result.htmlfile contains the following content:



```
<!DOCTYPE html>
<html>
<head>
	<title></title>
    <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/styles.css') }}">
</head>
<body>

	<header>
		<div class="container">
		<div id="brandname">
			ML App
		</div>
		<h2>Spam Detector For SMS Messages</h2>
		
	</div>
	</header>
	<p style="color:blue;font-size:20;text-align: center;"><b>Results for Comment</b></p>
	<div class="results">
	{% if prediction == 1%}
	<h2 style="color:red;">Spam</h2>
	{% elif prediction == 0%}
	<h2 style="color:blue;">Not a Spam (It is a Ham)</h2>
	{% endif %}
	</div>
</body>
</html>
```



From result.htm we can see that some code using syntax not normally found in HTML files: {% if prediction ==1%},{% elif prediction == 0%},{% endif %}This is jinja syntax, and it is used to access the prediction returned from our HTTP request within the HTML file.

We are almost there!

Once you have done all of the above, you can start running the API by either double click appy.py , or executing the command from the Terminal:

In [0]:
cd SMS-Message-Spam-Detector
python app.py

Now you could open a web browser and navigate to http://127.0.0.1:5000/, we should see a simple website with the content

Congratulations! We have now created an end-to-end machine learning (NLP) application at zero cost. If you look it back, the overall process is not complicated at all. With a little bit patience and desire to learn, anyone can do it. All the open-source tools make every thing possible.